In [ ]:
PROFIT

In [23]:
import utils.csv_utils as csv_utils 
import utils.dir_utils as dir_utils
import utils.dict_utils as dict_utils 
import utils.ptr_utils as ptr_utils
import utils.constants as constants 
import helpers.official as official
import helpers.search as search
import helpers.congress as congress
import pandas as pd 
from scipy.stats.mstats import gmean
import random 

In [22]:
# randomly give it one of the upper ones!!!
print(gmean([1000, 15000]))

print(gmean(range(1000, 15000)))

3872.9833462074166
6695.165084309057


In [27]:
_, input_df = dir_utils.get_data(combined=True)

# s = set()

# for _,t in input_df.iterrows():      
#     if t[constants.AMOUNT] == '$1,001 -':
#             s.add(ptr_utils.get_year(t[constants.TDATE]))
#         # n = official.get_name(t)
#         # s.add( (n, search.congress_gov_get(n, party_only=True), search.congress_gov_get(n, state_only=True))) 
    

# print(s)
        # {
        # '$1,000 - $15,000'
        # '$1,001 - $15,000',
        # '$1,001 -', 
        # '$15,000 - $50,000',
        # '$15,001 - $50,000',
        # '$50,001 - $100,000',
        # '$100,001 - $250,000',
        # '$250,001 - $500,000',
        # '$500,001 - $1,000,000',
        # '$1,000,000 +',
        # '$1,000,001 - $5,000,000',
        # '$5,000,001 - $25,000,000',
        # '$25,000,001 - $50,000,000',
        # 'Over $50,000,000', 
        # '$50,000,000 +',

        
        #  'Unknown', 
        #  }
        

# # t[constants.AMOUNT]

for x in ALL_OF_THE_POSSIBLE_ENTRIES_FOR_AMOUNT:
        print(x, get_gmean(x))

$1,000 - $15,000 6695.5508358990155
$1,001 - $15,000 6696.460269795503
$15,000 - $50,000 30815.197062653664
$15,001 - $50,000 30815.830944625086
$50,001 - $100,000 73576.3982240245
$100,001 - $250,000 169410.4485516184
$250,001 - $500,000 367879.95116074546
$500,001 - $1,000,000 735759.3923321357
$1,000,001 - $5,000,000 2750540.9233132955
$5,000,001 - $25,000,000 13752702.40315505
$25,000,001 - $50,000,000 36787944.627133004
Over $50,000,000 73575888.17583919
$50,000,000 + 73575888.17583919
Unknown None
$1,001 - 369.4916634719598
$1,000,000 + 36787944.627133004


In [ ]:
_, input_df = dir_utils.get_data(combined=True)
_, house_input_df = dir_utils.get_data(house=True)
_, senate_input_df = dir_utils.get_data(senate=True)